In [1]:
import requests
from multiprocessing import Process, Manager
import json
import pandas as pd

In [2]:
def get_story(story_id, stories):
    url = 'https://hacker-news.firebaseio.com/v0/item/%d.json' % story_id
    resp = requests.get(url)
    stories.append(resp.json())

In [3]:
def get_top_stories():
    manager = Manager()
    stories = manager.list()
    url = 'https://hacker-news.firebaseio.com/v0/topstories.json'
    ids = requests.get(url)
    processes = [Process(target=get_story, args=(sid, stories))
                 for sid in ids.json()]
    for p in processes:
        p.start()
    for p in processes:
        p.join()
    return stories

In [4]:
df = pd.read_json('../../data/hn.json')

In [5]:
df['id'] = df.index.astype(int)

In [6]:
top_stories = get_top_stories()

In [7]:
new_df = pd.DataFrame([t for t in top_stories])

In [8]:
joined = df.append(new_df)

In [9]:
joined.set_index('id').index

Int64Index([11693122, 11693168, 11693169, 11693184, 11693185, 11693193,
            11693217, 11693232, 11693240, 11693296,
            ...
            11722573, 11707449, 11711640, 11713423, 11708769, 11718494,
            11717710, 11717907, 11720912, 11707805],
           dtype='int64', name='id', length=1174)

In [10]:
df = joined.drop_duplicates(subset='id', keep='last')

In [11]:
df.set_index('id').to_json('../../data/hn.json')